The more I need to make graphs and charts the more I find myself not wanting to use high-level interfaces to do it.  Usually this is be

I need to make a clustering heatmap for work and the only ones I could find were either done in 

In [1]:
import pandas as pd
import numpy as np

In [2]:
colors_df = pd.read_csv('data/colors.csv', index_col=0)

df = pd.read_csv('data/2016_teams_standard_batting_8_31_16.csv', index_col=0)

In [3]:
colors_df = colors_df.loc[df.index]

In [4]:
# scale the data

df_std = (df - df.min(axis=0)) / (df.max(axis=0) - df.min(axis=0))
df_scaled = df_std * (1.0 - 0.0) + 0.0

In [5]:
# clean up the frames a little

names = colors_df['Teams']

In [6]:
colors_df.drop(['Teams', 'Abbrv.1'], axis=1, inplace=True)

In [7]:
def color_gen(tm):
    """ function to generate a random color from a given team thats not white"""
    from random import randint
    color = '#FFFFFF'
    while color == '#FFFFFF':
        color = colors_df.loc[tm][randint(0, len(colors_df.loc[tm].dropna())-1)]
    return color

In [8]:
from functools import partial

In [9]:
# create dictionary to generate random color for team

color_map = dict([(tm, partial(color_gen, tm)) for tm in colors_df.index.tolist()])

In [10]:
# clean up data an convert to ColumnDataSource for plotting
tms = []
batting = []
xs = []
ys = []
colors = []
alpha = []
value = []


from sklearn.metrics.pairwise import pairwise_distances
from scipy.cluster.hierarchy import linkage, dendrogram

X = pairwise_distances(df.values, metric='euclidean')

Z = linkage(X, 'ward')

In [11]:
results = dendrogram(Z, no_plot=True)
icoord, dcoord = results['icoord'], results['dcoord']
labels = list(map(int, results['ivl']))
df = df.iloc[labels]
df_scaled = df_scaled.iloc[labels]

In [12]:
for i, tm in enumerate(df.index):
    tms = tms + [tm]*len(df.columns)
    batting = batting + df.columns.tolist()
    xs = xs + list(np.arange(0.5, len(df.columns)+0.5))
    ys = ys + [i+0.5]*len(df.columns)
    colors = colors + [color_map[tm]() for i in range(len(df.columns))]
    value = value + df.loc[tm].tolist()
    alpha = alpha + df_scaled.loc[tm].tolist()


In [13]:
from bokeh.plotting import figure, output_file
from bokeh.models.sources import ColumnDataSource
from bokeh.models import HoverTool, ResizeTool, SaveTool

data = pd.DataFrame(dict(
        tms=tms,
        batting=batting,
        xs=xs,
        ys=ys,
        colors=colors,
        value=value,
        alpha=alpha
              ))

source = ColumnDataSource(data)

hover = HoverTool()

hover.tooltips = [
    ("Team", "@tms"),
    ("Batting Stat", ("@batting: @value")),
    ("Value", "@value")
]

height, width = df.shape

icoord = pd.DataFrame(icoord)
icoord = icoord * (data['ys'].max() / icoord.max().max())
icoord = icoord.values

dcoord = pd.DataFrame(dcoord)
dcoord = dcoord * (data['xs'].max() / dcoord.max().max())
dcoord = dcoord.values


hm = figure(x_range=[-40, 40],
            #y_range=[0, icoord.max()],
            height=800,
            width=800,
            tools=[ 
                    ResizeTool(), 
                    SaveTool()],
            webgl=True
)

for i, d in zip(icoord, dcoord):
    d = list(map(lambda x: -x, d))
    hm.line(x=d, y=i, line_color='black')

hm.add_tools(hover)


hm.rect(x='xs', y='ys',
        height=1,
        width=1,
        fill_color='colors',
        line_color='black',
        source=source,
        line_alpha=0.2,
        fill_alpha='alpha'
        
        )
hm.text([data['xs'].max()+0.51] * len(data['tms'].unique()), 
        data['ys'].unique().tolist(), 
        text=[nm for nm in names.iloc[labels]],
        text_baseline='middle',
        text_font_size='6pt'
       
       )

hm.axis.major_tick_line_color = None
hm.axis.minor_tick_line_color = None
hm.axis.major_label_text_color = None
hm.axis.major_label_text_font_size = '0pt'
hm.axis.axis_line_color = None
hm.grid.grid_line_color = None
hm.outline_line_color = None

In [14]:
from bokeh.plotting import output_notebook, show

In [17]:
output_notebook()


Loading BokehJS ...

In [18]:
show(hm)